## Logic to upload GHArchive Files

Let us go through the logic to upload GHArchive website to HDFS.
* Make sure the folder is created to place the files.
* Get the content of the file using `requests` module using`requests.get`. We need to pass the URI from which the files are supposed to be downloaded.
* It will return an object of type **Response** which contain the `content` attribute.
* The content can be uploaded to any file system. As our cluster is not configured with Web HDFS, we will first add files to local file system, then we will upload to HDFS using `hdfs dfs -put` command.

* Creating the base directory.

In [ ]:
!hdfs dfs -rm -R -skipTrash /user/${USER}/itv-github/streaming/landing/ghactivity

In [ ]:
!hdfs dfs -mkdir -p /user/${USER}/itv-github/streaming/landing/ghactivity

* Get the content of the file.

In [ ]:
import requests

In [ ]:
file = '2021-01-13-0.json.gz'

In [ ]:
res = requests.get(f'https://data.gharchive.org/{file}')

In [ ]:
res

In [ ]:
type(res.content)

* Uploading to local file system.

In [ ]:
!rm -rf /home/${USER}/data/ghactivity

In [ ]:
!mkdir -p /home/${USER}/data/ghactivity

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
year = file[:4]
year

In [ ]:
month = file[5:7]
month

In [ ]:
dayofmonth = file[8:10]
dayofmonth

In [ ]:
target_local_folder = f'/home/{username}/data/ghactivity/year={year}/month={month}/dayofmonth={dayofmonth}'

In [ ]:
target_local_folder

In [ ]:
import subprocess
subprocess.check_call(f'mkdir -p {target_local_folder}', shell=True)

In [ ]:
target_file = open(f'{target_local_folder}/{file}', 'wb')

In [ ]:
upload_res = target_file.write(res.content)

In [ ]:
upload_res

In [ ]:
target_file.close()

In [ ]:
!ls -lR /home/${USER}/data/ghactivity/

* Copy the files to HDFS and validate.

In [ ]:
target_hdfs_folder = f'/user/{username}/itv-github/streaming/landing/ghactivity/year={year}/month={month}/dayofmonth={dayofmonth}'

In [ ]:
target_hdfs_folder

In [ ]:
subprocess.check_call(f'hdfs dfs -mkdir -p {target_hdfs_folder}', shell=True)

In [ ]:
subprocess.check_call(f'hdfs dfs -put {target_local_folder}/{file} {target_hdfs_folder}', shell=True)

In [ ]:
!hdfs dfs -ls -R /user/${USER}/itv-github/streaming/landing/ghactivity